In [ ]:
%pylab inline

# JSAnimation import available at https://github.com/jakevdp/JSAnimation
from JSAnimation import IPython_display
from matplotlib import animation

import numpy as np

from ndio.remote.boss.remote import Remote
from ndio.ndresource.boss.resource import *
from ndio.convert.tiff import load_tiff_multipage

import os
from datetime import datetime
from skimage.filters.rank import median
from skimage.morphology import disk

from skimage import exposure
from skimage.filters.rank import enhance_contrast_percentile
from skimage.morphology import disk


data_dir = "/Users/kleisdm1/my_data/MICrONS/data"

## Setup ndio Remote instance for the Boss

We are passing in a config file that is set up with the endpoint location and credentials

In [ ]:
rmt = Remote('demo.cfg')
API_VER = 'v0.5'

## Create Resources to for Ingest Job
Currently you must pre-create resources (collection, experiment, channel) for ingest.  This specifies _were_ data will be written in the Boss.

In [ ]:
col_name = "col5"
exp_name = "exp5"
ch_name = "ch5"

In [ ]:
# Create Collection
col = CollectionResource(col_name, API_VER, 'Ingest Example Collection')
try:
    col = rmt.project_create(col)
    if col:
        print("Successfully Created Collection {}".format(col_name))
except:
    col = rmt.project_get(col)


# Create functional Coordinate Frame
coord_ca = CoordinateFrameResource('functional_test_coord_frame', API_VER,
                                       'Coordinate frame for a sample calcium imaging data set.', 0, 796, 0, 512, 0, 1)
try:
    coord_ca = rmt.project_create(coord_ca)
    if coord_ca:
        print("Successfully Created Coordinate Frame 'functional_test_coord_frame'")
except:
    coord_ca = rmt.project_get(coord_ca)


# Create Experiment for EM data
exp = ExperimentResource(exp_name, col_name, API_VER, 'Experiment for a test functional dataset.',
                            coord_ca.id, max_time_sample=1600)
try:
    exp = rmt.project_create(exp)
    if exp:
        print("Successfully Created Experiment {}".format(exp_name))
except:
    exp = rmt.project_get(exp)

dynamic_chan = ChannelResource(ch_name, col_name, exp_name, API_VER,
                                   'CaMP6f calcium channel (green channel)',
                                   datatype='uint16')
try:
    dynamic_chan = rmt.project_create(dynamic_chan)
    if dynamic_chan:
        print("Successfully Created Channel {}".format(ch_name))
except:
    dynamic_chan = rmt.project_get(dynamic_chan)

## Acessing Calcium Imaging Data - Multichannel uint16 with time series support

A small sample dataset provided by Sandra Kuhlman of CMU has been loaded into the Boss.  It is 796x512x1 spatially with 1500 time samples.  It also contains two channels that store different information

In [ ]:
import pprint
pp = pprint.PrettyPrinter(indent=4)

calcium_experiment_channels = rmt.project_list(exp)
pp.pprint(calcium_experiment_channels)

In [ ]:
# Do a cutout
x_rng = '0:796'
y_rng = '0:512'
z_rng = '0:1'
t_rng = '0:25'
print("Downloading data")
start = datetime.now()
time_data = rmt.cutout_get(dynamic_chan, 0, x_rng, y_rng, z_rng, t_rng)
print("Done!")
print(time_data.shape)
print(time_data.min())
print(time_data.max())

In [ ]:
# Method to window a 16-bit image
def window_img(image, min_val, max_val):
    image_scaled = zeros(image.shape)
    np.clip(image, min_val, max_val, out=image_scaled)
    np.subtract(image_scaled, min_val, out=image_scaled)
    #return np.multiply( image_scaled,np.iinfo(np.uint16).max/(max_val-min_val),out=image_scaled)
    return np.multiply(image_scaled,255./(max_val-min_val),out=image_scaled).astype(np.uint8)

# Window dynamic channel cutout
time_data_dynamic = window_img(time_data, 3500, 7800)

# Denoise
time_data_dynamic_denoise = np.zeros(time_data.shape, dtype=uint8)
for frame in range(0, time_data.shape[0]-1):
    time_data_dynamic_denoise[frame, 0, :, :] = (median(time_data_dynamic[frame,0, :, :], disk(3)))



In [ ]:
fig = plt.figure()
im = plt.imshow(time_data_dynamic_denoise[1,0,:,:], cmap=plt.cm.gray, interpolation='bicubic') 

def init():
    im.set_data(time_data_dynamic_denoise[1,0,:,:])

def animate(i): 
    im.set_data(time_data_dynamic_denoise[i,0,:,:])
    return im

animation.FuncAnimation(fig, animate, init_func=init, frames=time_data_dynamic_denoise.shape[0],
                               interval=75)

